In [3]:
import pandas as pd

file_path = 'data/All-Purpose Cleaners.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Now df is a DataFrame containing the data from your Excel file

df_dict = df.to_dict()

df.to_csv('data/All-Purpose Cleaners.csv', index=False)


In [15]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(
    dotenv_path='ops/.env',
)

api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=api_key)



In [4]:
file = client.files.create(
  file=open("data/All-Purpose Cleaners.csv", "rb"),
  purpose='assistants'
)

print(file.id)

file-tOeMTvnTeIwv1n4mYa7t3fwf


In [5]:
assistant = client.beta.assistants.create(
  instructions="You are a data analyst assistant when given a requirement and a csv file analyse the csv file and answer if the requirement is satisfied.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)

In [9]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": """This is the requirement. 1. Product name must include the brand name, actual name and size of the product as part
of the overall name e.g. “LaxmiBhog atta 1KG bag” 2. Every product must have high quality images (preferably 500x500 px) 3. Each product must have meaningful and related short description column and full description column associated
with the product 4. Slab options will be shown in a separate slab table for those products who have more than one pricing slab 5. All variants of a product will be shown in a variants section sorted by variant name in a horizontal scrollable section
""", "file_ids": [file.id]
        }
    ]
)

In [10]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

print(run.status)

queued


In [12]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run.status)

completed


In [13]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    )

print(messages.data[0].content[0].text.value)

Based on the initial analysis of the CSV file, the following observations can be made:

1. The `Product Name` includes information that likely corresponds to the brand name, actual name, and size, as products have names like "Harpic Disinfectant Toilet Cleaner Liquid, Original - 500 ml" which satisfy the naming convention requirement.
2. The products have image URLs provided in numerous 'Images' columns, indicating that high-quality images are potentially present. However, without accessing the actual images, we can't confirm if they are 500x500 pixels as required.
3. There are columns that seem to pertain to product descriptions, including 'Product Description', 'Features', 'Specification', and 'About'. Some products have meaningful descriptions, which suggests that the third requirement may be satisfied in at least some cases.

The current data provided doesn't directly include information about slab pricing options or a separated section for product variants. Therefore, we cannot co